In [154]:
import pandas as pd
import numpy as np

In [155]:
titanic = pd.read_csv('train.csv')
t = titanic.set_index(['PassengerId'])
pd.set_option('display.max_rows', None)

In [156]:
def fit(dataset, n_count):
    result = pd.DataFrame(index=t.columns, columns=t.columns)
    for outer_column in t.columns:
        sum = 0
        if outer_column in ['Name', 'Survived']:
            continue
        ff = t[outer_column].value_counts()[t[outer_column].value_counts() > n_count]
        if len(ff) < 1:
            continue
        #ff = t[t[outer_column].duplicated(keep=False)][outer_column].value_counts()
        for i in ff.items():
            percent = round(t[t[outer_column] == i[0]].fillna(-1)['Survived'].value_counts(normalize=True)*100, 3).max()
            column_value = t[t[outer_column] == i[0]].fillna(-1)['Survived'].value_counts().sort_values(ascending=False).head(1).index[0]
            sum += i[1] * percent
        result[outer_column]['Survived'] = sum / ff.sum()
    
    t_result = result.dropna(axis='columns', how='all').dropna(axis='index', how='all')#.sort_values(by='Survived', ascending=False)
    return t_result

# res = []
# for i in range(5):
#     res.append(fit(i))
# pd.concat(res).sort_values(['Ticket', 'Sex', 'Fare', 'Pclass', 'Embarked', 'Age', 'SibSp', 'Parch'], ascending=False)
#fit(4).transpose().sort_values(by='Survived')

In [157]:
def Search_max(entry, sort_columns, dataset):
    for column in sort_columns.index: # ['Ticket', 'Cabin', 'Sex', 'Fare', 'Pclass', 'Age', 'Embarked', 'SibSp','Parch']
        for value in t[column]: # t["Ticket"]: [1061, A/5 21171 , ....]
            if value == entry[column]: # if 1061 == entry['Ticket']
                percent = round(t[t[column] == value].fillna(-1)['Survived'].value_counts(normalize=True)*100, 2).max()
                column_value = t[t[column] == value].fillna(-1)['Survived'].value_counts().sort_values(ascending=False).head(1).index[0]
                return column, percent, column_value

def Predict(x, sort_columns, dataset):
    entries = []
    for entry in x:
        entries.append({
            'Pclass': entry[1],
            'Name': entry[2],
            'Sex': entry[3],
            'Age': entry[4],
            'SibSp': entry[5],
            'Parch': entry[6],
            'Ticket': entry[7],
            'Fare': entry[8],
            'Cabin': entry[9],
            'Embarked': entry[10]
        })

    response = []
    result = []
    for entry in entries:
        response.append(Search_max(entry, sort_columns, dataset))

    for i in response:
        result.append(i[2])

    return result

In [158]:
dataset = pd.read_csv('test.csv')
dataset.head()

dataset2 = pd.read_csv('gender_submission.csv')
dataset2.head(), dataset.head()

dataset_final = dataset.merge(dataset2, on='PassengerId')

x = dataset_final.iloc[:, :-1].values
y = dataset_final.iloc[:, 11].values


In [175]:
n_count = 3
dataset = t
sort_columns = fit(dataset, n_count).transpose().sort_values(by='Survived', ascending=False)
sort_columns

,Survived
Ticket,89.247344
Sex,78.675588
Fare,76.716935
Pclass,67.901302
Cabin,66.666667
Embarked,63.779252
SibSp,63.30008
Age,63.287278
Parch,63.033575


In [179]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

y_pred = Predict(x_test, sort_columns, dataset)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)

print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[73  8]
 [13 32]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.90      0.87        81
           1       0.80      0.71      0.75        45

    accuracy                           0.83       126
   macro avg       0.82      0.81      0.81       126
weighted avg       0.83      0.83      0.83       126

Accuracy: 0.8333333333333334
